In [1]:
from IPython.core.display import HTML 

# Introduction to MongoDB with PyMongo and NOAA Data

This notebook provides a basic walkthrough of how to use MongoDB and is based on a tutorial originally by [Alberto Negron](http://altons.github.io/python/2013/01/21/gentle-introduction-to-mongodb-using-pymongo/).

Metadata records are frequently stored as JSON, and almost anything you get from an API will be JSON. For example, check out the [metadata records](https://data.noaa.gov/data.json) for the National Oceanic and Atmospheric Administration. 

MongoDB is a great tool to use with JSON data because it stores structured data as JSON-like documents, using dynamic schemas (called BSON), rather than predefined schemas. 

In MongoDB, an element of data is called a document, and documents are stored in collections. One collection may have any number of documents. Collections are a bit like tables in a relational database, and documents are like records. But there is one big difference: every record in a table has the same fields (with, usually, differing values) in the same order, while each document in a collection can have completely different fields from the other documents.

Documents are Python dictionaries that can have strings as keys and can contain various primitive types (int, float,unicode, datetime) as well as other documents (Python dicts) and arrays (Python lists).

## Getting started
First we need to import `json` and `pymongo`.

Note that the `pprint` module provides a capability to “pretty-print” arbitrary Python data structures in a form which can be used as input to the interpreter. This is particularly helpful with JSON. You can read more about `pprint` [here](https://docs.python.org/2/library/pprint.html).

In [2]:
import json
import pymongo
from pprint import pprint

## Connect    
Just as with the relational database example with `sqlite`, we need to begin by setting up a connection. With MongoDB, we will be using `pymongo`, though MongoDB also comes with a [console API that uses Javascript](https://docs.mongodb.org/manual/tutorial/write-scripts-for-the-mongo-shell/).    


To make our connection, we will use the PyMongo method `MongoClient`:

In [3]:
conn=pymongo.MongoClient()
conn

MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True)

## Create and access a database    
Mongodb creates databases and collections automatically for you if they don't exist already. A single instance of MongoDB can support multiple independent databases. When working with PyMongo, we access databases using attribute style access, just like we did with `sqlite`:

In [4]:
db = conn.mydb
db

Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'mydb')

## Collections    
A collection is a group of documents stored in MongoDB, and can be thought of as roughly the equivalent of a table in a relational database. Getting a collection in PyMongo works the same as getting a database:

In [5]:
collection = db.my_collection
collection

Collection(Database(MongoClient(host=['localhost:27017'], document_class=dict, tz_aware=False, connect=True), u'mydb'), u'my_collection')

## Insert data   
To insert some data into MongoDB, all we need to do is create a dict and call `insert_one` on the collection object:

In [6]:
doc = {"class":"XBUS-502","date":"03-05-2016","instructor":"Bengfort","classroom":"C222","roster_count":"25"}
collection.insert_one(doc)

In [7]:
conn.database_names()

[u'local', u'mydb']

In [8]:
db.collection_names()

[u'my_collection', u'system.indexes']

## A practical example

Let's say you wanted to gather up a bunch of JSON metadata records and store them for analysis. 

```python
import requests

NOAA_URL = "https://data.noaa.gov/data.json"

def load_data(URL):
    """
    Loads the data from URL and returns data in JSON format.
    """
    r = requests.get(URL)
    data = r.json()
    return data
    
noaa = load_data(NOAA_URL)
```

This takes a long time, so I've created a file for you that contains a small chunk of the records to use for today's workshop.

In [9]:
with open("data_sample.json") as data_file:    
    noaa = json.load(data_file)

In [10]:
len(noaa)

1722

### Checking out the data
Now let's print out just one record to examine the structure. 

In [11]:
pprint(noaa[0])

{u'@type': u'dcat:Dataset',
 u'accessLevel': u'public',
 u'accrualPeriodicity': u'irregular',
 u'bureauCode': [u'006:48'],
 u'contactPoint': {u'@type': u'vcard:Contact',
                   u'fn': u'Ilya P. Romanov',
                   u'hasEmail': u'mailto:nsidc@nsidc.org'},
 u'description': u"This data set contains sea ice and snow measurements collected during aircraft landings associated with the Soviet Union's historical Sever airborne and North Pole drifting station programs. The High-Latitude Airborne Annual Expeditions Sever (Sever means North) took place in 1937, 1941, 1948-1952, and 1954-1993 (Konstantinov and Grachev, 2000). In Spring 1993, the last (45th) Sever expedition finished long-term activity in the Arctic. Snow and sea ice data were collected, along with meteorological and hydrological measurements (the latter are not part of this data set). Up to 202 landings were accomplished each year.  The data set contains measurements of 23 parameters, including ice thickness a

Or say we wanted just the "description" field:

In [12]:
pprint(noaa[0]['description'])

u"This data set contains sea ice and snow measurements collected during aircraft landings associated with the Soviet Union's historical Sever airborne and North Pole drifting station programs. The High-Latitude Airborne Annual Expeditions Sever (Sever means North) took place in 1937, 1941, 1948-1952, and 1954-1993 (Konstantinov and Grachev, 2000). In Spring 1993, the last (45th) Sever expedition finished long-term activity in the Arctic. Snow and sea ice data were collected, along with meteorological and hydrological measurements (the latter are not part of this data set). Up to 202 landings were accomplished each year.  The data set contains measurements of 23 parameters, including ice thickness and snow depth on the runway and surrounding area; ridge, hummock, and sastrugi dimensions and areal coverage; and snow density. The sea ice thickness data are of particular importance, as ice thickness measurements for the Arctic Basin are scarce. These data are a subset of those used to crea

### Define the database
We will want to enter these records into our database. But first, we'll define a specific database for the NOAA records:

In [13]:
db = conn.earthwindfire

### Define the collection
Next we define the collection where we'll insert the NOAA metadata records:

In [14]:
records = db.records

### Insert data    
Then we loop through each record in the NOAA dataset and insert just the target information for each into the collection:

In [15]:
def insert(metadata):
    for dataset in metadata:
        data ={}
        data["title"] = dataset["title"]
        data["description"] = dataset["description"]
        data["keywords"] = dataset["keyword"]
        data["accessLevel"] = dataset["accessLevel"]
        data["lang"] = dataset["language"]
        # choose your own
        # choose your own
        # choose your own 
        # choose your own

        records.insert_one(data)

insert(noaa)

In [16]:
records.count()

1722

### Querying 

#### Querying with `.findOne( )`    
The find_one() method selects and returns a single document from a collection and returns that document (or None if there are no matches). It is useful when you know there is only one matching document, or are only interested in the first match

In [17]:
records.find_one()

{u'_id': ObjectId('56d9a92abe18b11a0849df1f'),
 u'accessLevel': u'public',
 u'description': u"This data set contains sea ice and snow measurements collected during aircraft landings associated with the Soviet Union's historical Sever airborne and North Pole drifting station programs. The High-Latitude Airborne Annual Expeditions Sever (Sever means North) took place in 1937, 1941, 1948-1952, and 1954-1993 (Konstantinov and Grachev, 2000). In Spring 1993, the last (45th) Sever expedition finished long-term activity in the Arctic. Snow and sea ice data were collected, along with meteorological and hydrological measurements (the latter are not part of this data set). Up to 202 landings were accomplished each year.  The data set contains measurements of 23 parameters, including ice thickness and snow depth on the runway and surrounding area; ridge, hummock, and sastrugi dimensions and areal coverage; and snow density. The sea ice thickness data are of particular importance, as ice thickness

#### Querying with `.find( )`
To get more than a single document as the result of a query we use the `find()` method. `find()` returns a Cursor instance, which allows us to iterate over all matching documents.

```python
records.find()
```

For example, we can iterate over the first 2 documents (there are a lot in the collection and this is just an example) in the records collection

In [18]:
for rec in records.find()[:2]:
    pprint(rec)

{u'_id': ObjectId('56d9a92abe18b11a0849df1f'),
 u'accessLevel': u'public',
 u'description': u"This data set contains sea ice and snow measurements collected during aircraft landings associated with the Soviet Union's historical Sever airborne and North Pole drifting station programs. The High-Latitude Airborne Annual Expeditions Sever (Sever means North) took place in 1937, 1941, 1948-1952, and 1954-1993 (Konstantinov and Grachev, 2000). In Spring 1993, the last (45th) Sever expedition finished long-term activity in the Arctic. Snow and sea ice data were collected, along with meteorological and hydrological measurements (the latter are not part of this data set). Up to 202 landings were accomplished each year.  The data set contains measurements of 23 parameters, including ice thickness and snow depth on the runway and surrounding area; ridge, hummock, and sastrugi dimensions and areal coverage; and snow density. The sea ice thickness data are of particular importance, as ice thickness

#### Searching 
MongoDB queries are represented as JSON-like structures just like documents. To build a query, you just need to specify a dictionary with the properties you want the results to match. For example, let's say we were just interested in publically available satellite data from [NESDIS](http://www.nesdis.noaa.gov/).

This query will match all documents in the records collection with keywords code "NESDIS".

In [19]:
records.find({"keywords": "NESDIS"}).count()

1117

1117 is probably more than we want to print out in a Jupyter Notebook...    

We can further narrow our search by adding additional fields

In [20]:
records.find({"keywords": "NESDIS","keywords": "Russia","accessLevel":"public"}).count()

2

Since there's only two, let's check them out:

In [21]:
for r in records.find({"keywords": "NESDIS","keywords": "Russia","accessLevel":"public"}):
    pprint(r)

{u'_id': ObjectId('56d9a92abe18b11a0849df20'),
 u'accessLevel': u'public',
 u'description': u'This data set was distributed by NSIDC until October, 2003, when it was withdrawn from distribution because it duplicates the NOAA National Climatic Data Center (NCDC) data set DSI-3720. The NCDC data set is revised and updated beyond what was distributed by NSIDC. This archive consists of monthly precipitation measurements from 622 stations located in the Former Soviet Union.',
 u'keywords': [u'Continent > Europe > Eastern Europe > Russia',
               u'EARTH SCIENCE > Atmosphere > Precipitation > Precipitation Amount',
               u'EARTH SCIENCE > Atmosphere > Precipitation > Precipitation Rate',
               u'EARTH SCIENCE > Atmosphere > Precipitation > Rain',
               u'EARTH SCIENCE > Atmosphere > Precipitation > Snow',
               u'EARTH SCIENCE > Terrestrial Hydrosphere > Snow/Ice > Snow Cover',
               u'EOSDIS > Earth Observing System Data Information Syste

### If you already know SQL...

The following table provides an overview of common SQL aggregation terms, functions, and concepts and the corresponding MongoDB aggregation operators:    
    
| SQL Terms, Functions, and Concepts  | MongoDB Aggregation Operators  |
| ----------------------------------  |:-------------------------------|
| WHERE                               | \$match                        |
| GROUP BY                            | \$group                        |
| HAVING                              | \$match                        |
| SELECT	                          | \$project                      |
| ORDER BY	                          | \$sort                         |
| LIMIT                               | \$limit                        |
| SUM()   	                          | \$sum                          |
| COUNT()	                          | \$sum                          |
| join	                              | \$lookup                       |

### Full text search with a text index
One of the things that makes MongoDB special is that it enables us to create search indexes. Indexes provide high performance read operations for frequently used queries.

In particular, a __text index__ will enable us to search for string content in a collection. _Keep in mind that a collection can have at most one text index._ 

We will create a text index on the description field so that we can search inside our NOAA records text:

In [23]:
db.records.create_index([('description', 'text')])

u'description_text'

To test our newly created text index on the description field, we will search documents using the `$text` operator. Let's start by looking for all the documents that have the word 'precipitation' in their description field. 

In [25]:
cursor = db.records.find({'$text': {'$search': 'precipitation'}})
for rec in cursor:
    print rec

{u'lang': [u'en-US'], u'_id': ObjectId('56d9a92abe18b11a0849df43'), u'description': u"Integrated Surface Data (ISD) is digital data set DSI-3505, archived at the National Climatic Data Center (NCDC). The ISD database is composed of worldwide surface weather observations from over 20,000 stations, collected and stored from sources such as the Automated Weather Network (AWN), the Global Telecommunications System (GTS), the Automated Surface Observing System (ASOS), and data keyed from paper forms. Most digital observations are decoded either at operational centers and forwarded to the Federal Climate Complex (FCC) in Asheville, NC, or decoded at the FCC. The US Air Force Combat Climatology Center (AFCCC), the National Climatic Data Center (NCDC), and the US Navy's Fleet Numerical Meteorological and Oceanographical Command Detachment (FNMOD), make up the FCC in Asheville. Each agency is responsible for data ingest, quality control, and customer support for surface climatological data. All

In [26]:
cursor = db.records.find({'$text': {'$search': 'fire'}})
cursor.count()

11

If we want to create a new text index, we can do so by first dropping the first text index:

In [27]:
db.records.drop_index("description_text") 

We can also create a __wildcard__ text index for scenarios where we want any text fields in the records to be searchable. In such scenarios you can index all the string fields of your document using the $** wildcard specifier.

The query would go something like this:

In [28]:
db.records.create_index([("$**","text")])

u'$**_text'

In [29]:
cursor = db.records.find({'$text': {'$search': "Russia"}})
for rec in cursor:
    pprint(rec)

{u'_id': ObjectId('56d9a92abe18b11a0849df36'),
 u'accessLevel': u'public',
 u'description': u'This film documents the activities that occurred on Drifting Station Alpha in the Arctic Ocean during the International Geophysical Year, 1957 to 1958. The film is narrated by project leader, Norbert Untersteiner, and chronicles the life of the team as they built their camp and set up experiments. Station Alpha was the first long-term scientific base on arctic pack ice operated by a Western country. At the time of its establishment, Russia had already operated six drifting ice camps of this kind. However, due to the strategic importance and sensitivity of the Arctic Basin, little information from these early stations had reached the West. The documentary was filmed and produced by Frans van der Hoeven (Senior Scientist at Station Alpha) and Norbert Untersteiner (Scientific Leader of Station Alpha). Station Alpha drifted in an area of the Arctic ocean located 500 km north of Barrow, Alaska USA 

### Projections 

Projections allow you to pass along the documents with only the specified fields to the next stage in the pipeline. The specified fields can be existing fields from the input documents or newly computed fields.

For example, let's redo our fulltext Russia search, but project just the titles of the records:

In [31]:
cursor = db.records.find({'$text': {'$search': "Russia"}}, {"title": 1,"_id":0 })
for rec in cursor:
    print rec

{u'title': u'International Geophysical Year, 1957-1958: Drifting Station Alpha Documentary Film'}
{u'title': u'Morphometric Characteristics of Ice and Snow in the Arctic Basin: Aircraft Landing Observations from the Former Soviet Union, 1928-1989'}
{u'title': u'Eastern Bering Sea Acoustic-Trawl Survey of Walleye Pollock (DY1006)'}
{u'title': u'Monthly Mean Precipitation Sums at Russian Arctic Stations, 1966-1990'}
{u'title': u'Temperature, salinity, and other data from buoy casts in the Arctic Ocean, Barents Sea and Beaufort Sea from 1948 to 1993 (NCEI Accession 9800040)'}
{u'title': u'Former Soviet Union Monthly Precipitation Archive, 1891-1993'}
{u'title': u'Russian River Ice Thickness and Duration'}
{u'title': u'Daily Precipitation Sums at Coastal and Island Russian Arctic Stations, 1940-1990'}
{u'title': u'Climate Prediction Center (CPC) East Atlantic/ Western Russia Teleconnection Pattern Index'}


### Limit    

`.limit()` passes the first _n_ documents unmodified to the pipeline where _n_ is the specified limit. For each input document, this method outputs either one document (for the first _n_ documents) or zero documents (after the first _n_ documents).

In [41]:
cursor = db.records.find({'$text': {'$search': "Russia"}}, {"title": 1,"_id":0 }).limit(2)
for rec in cursor:
    print rec

{u'title': u'International Geophysical Year, 1957-1958: Drifting Station Alpha Documentary Film'}
{u'title': u'Morphometric Characteristics of Ice and Snow in the Arctic Basin: Aircraft Landing Observations from the Former Soviet Union, 1928-1989'}


### Aggregate
MongoDB can perform aggregation operations with `.aggregate()`, such as grouping by a specified key and evaluating a total or a count for each distinct group.    

Use the `$group` stage to group by a specified key using the \_id field. `$group` accesses fields by the field path, which is the field name prefixed by a dollar sign.    

For example, we can use `$group` to aggregate all the languages of the NOAA records:

In [43]:
cursor = db.records.aggregate(
    [
        {"$group": {"_id": "$lang", "count": {"$sum": 1}}}
    ]
)
for document in cursor:
    pprint(document)

{u'_id': [u'en-US'], u'count': 1722}


Or we can combine `$match` and `$group` to aggregate the titles of just the public access records that match the word 'Soviet':

In [44]:
cursor = db.records.aggregate(
    [
        {"$match": {'$text': {'$search': "Russia"}, "accessLevel": "public"}},
        {"$group": {"_id": "$title"}}
    ]
)

for document in cursor:
    pprint(document)

{u'_id': u'Climate Prediction Center (CPC) East Atlantic/ Western Russia Teleconnection Pattern Index'}
{u'_id': u'Daily Precipitation Sums at Coastal and Island Russian Arctic Stations, 1940-1990'}
{u'_id': u'Russian River Ice Thickness and Duration'}
{u'_id': u'Former Soviet Union Monthly Precipitation Archive, 1891-1993'}
{u'_id': u'Temperature, salinity, and other data from buoy casts in the Arctic Ocean, Barents Sea and Beaufort Sea from 1948 to 1993 (NCEI Accession 9800040)'}
{u'_id': u'Eastern Bering Sea Acoustic-Trawl Survey of Walleye Pollock (DY1006)'}
{u'_id': u'Morphometric Characteristics of Ice and Snow in the Arctic Basin: Aircraft Landing Observations from the Former Soviet Union, 1928-1989'}
{u'_id': u'Monthly Mean Precipitation Sums at Russian Arctic Stations, 1966-1990'}
{u'_id': u'International Geophysical Year, 1957-1958: Drifting Station Alpha Documentary Film'}


### Length    
find all that have this length, emphasis nested data structures, can’t do it in relational

### The aggregation pipeline 

The [aggregation pipeline](https://docs.mongodb.org/manual/core/aggregation-pipeline/) allows MongoDB to provide native aggregation capabilities that corresponds to many common data aggregation operations in SQL.  Here's where you will put the pieces together to aggregate to get results that you can begin to analyze and perform machine learning on.

In [ ]:
from IPython.display import Image
Image(filename='images/mongodb_pipeline.png', width=600, height=300)

### Removing data    

It's easy (almost too easy) to delete projects, collections, and databases in MongoDB. Before we get rid of anything, let's determine what collections we have in our database:

In [ ]:
conn.earthwindfire.collection_names()

Now let's delete our records collection and check again to see what collections are in our database:

In [ ]:
conn.earthwindfire.drop_collection("records")
conn.earthwindfire.collection_names()

We can also just drop a database. First let's determine what databases we have:

In [ ]:
conn.database_names()

Now let's remove the earthwindfire database:

In [ ]:
conn.drop_database("earthwindfire")
conn.database_names()

Nice work!

## Miscellaneous

### Statistics    

The [`dbstats`](https://docs.mongodb.org/manual/reference/method/db.stats/) method returns statistics that reflect the use state of a single database:

In [45]:
db.command({'dbstats': 1})

{u'avgObjSize': 2284.814814814815,
 u'collections': 3,
 u'dataFileVersion': {u'major': 4, u'minor': 22},
 u'dataSize': 3948160.0,
 u'db': u'earthwindfire',
 u'extentFreeList': {u'num': 0, u'totalSize': 0},
 u'fileSize': 67108864.0,
 u'indexSize': 5322576.0,
 u'indexes': 2,
 u'nsSizeMB': 16,
 u'numExtents': 8,
 u'objects': 1728,
 u'ok': 1.0,
 u'storageSize': 11194368.0}

[`collStats`](https://docs.mongodb.org/manual/reference/command/collStats/) returns a variety of storage statistics for a given collection. Let's try it out for our NOAA records collection:

In [46]:
db.command({'collstats': 'records', 'verbose': 'true' })

{u'avgObjSize': 2292,
 u'capped': False,
 u'count': 1722,
 u'extents': [{u'len': 8192, u'loc: ': {u'file': 0, u'offset': 20480}},
  {u'len': 32768, u'loc: ': {u'file': 0, u'offset': 167936}},
  {u'len': 131072, u'loc: ': {u'file': 0, u'offset': 200704}},
  {u'len': 524288, u'loc: ': {u'file': 0, u'offset': 331776}},
  {u'len': 2097152, u'loc: ': {u'file': 0, u'offset': 856064}},
  {u'len': 8388608, u'loc: ': {u'file': 0, u'offset': 2953216}}],
 u'indexDetails': {},
 u'indexSizes': {u'$**_text': 5257168, u'_id_': 65408},
 u'lastExtentSize': 8388608.0,
 u'nindexes': 2,
 u'ns': u'earthwindfire.records',
 u'numExtents': 6,
 u'ok': 1.0,
 u'paddingFactor': 1.0,
 u'paddingFactorNote': u'paddingFactor is unused and unmaintained in 3.0. It remains hard coded to 1.0 for compatibility only.',
 u'size': 3947616,
 u'storageSize': 11182080,
 u'totalIndexSize': 5322576,
 u'userFlags': 1}